# High Frequency Trading Algorithm

You have been tasked by the investment firm Renaissance High Frequency Trading (RHFT) to develop an automated trading strategy utilizing a combination of machine learning algorithms and high frequency algorithms. RHFT wants this new algorithm to be based on stock market data of the 30 stocks in the Dow Jones at the minute level and to conduct buys and sells every minute based on 1 min, 5 min, and 10 min Momentum. The CIO asked you to choose the Machine Learning Algorithm best suited for this task and wants you to execute the trades via Alpaca's API.

### Initial Set-Up

In [1]:
import os
from pathlib import Path
import alpaca_trade_api as tradeapi
import pandas as pd
import numpy as np
import datetime
import time
from dotenv import load_dotenv


In [2]:
# Load .env enviroment variables
# YOUR CODE HERE

load_dotenv()


True

In [3]:
# Set Alpaca API key and secret
# YOUR CODE HERE

ALPACA_API_KEY = os.getenv('ALPACA_API_KEY')
ALPACA_SECRET_KEY = os.getenv('ALPACA_SECRET_KEY')
base_url = 'https://paper-api.alpaca.markets'

print(type(ALPACA_API_KEY))
print(type(ALPACA_SECRET_KEY))
print(type(base_url))

<class 'str'>
<class 'str'>
<class 'str'>


In [4]:
# Create the Alpaca API object, specifying use of the paper trading account:
# YOUR CODE HERE

api = tradeapi.REST(
    ALPACA_API_KEY,
    ALPACA_SECRET_KEY,
    base_url,
    api_version="v2")

In [5]:
# obtain account information
account = api.get_account()
print(account)

Account({   'account_blocked': False,
    'account_number': 'PA32IVXERF1F',
    'accrued_fees': '0',
    'buying_power': '200000',
    'cash': '100000',
    'created_at': '2022-01-22T00:56:46.046436Z',
    'crypto_status': 'ACTIVE',
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '100000',
    'id': 'e3535156-a8cc-4e05-b336-0e8221a49d85',
    'initial_margin': '0',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '0',
    'pattern_day_trader': False,
    'pending_transfer_in': '0',
    'portfolio_value': '100000',
    'regt_buying_power': '200000',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '100000',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': False})


## Part 2: Train and Compare Multiple Machine Learning Algorithms

 In this section, you'll train each of the requested algorithms and compare performance. Be sure to use the same parameters and training steps for each model. This is necessary to compare each model accurately.

### Preprocessing Data

#### 1. Generate your feature data (`X`) and target data (`y`):
* Create a dataframe `X` that contains all the columns from the returns dataframe that will be used to predict `F_1_m_returns`.
* Create a variable, called `y`, that is equal 1 if `F_1_m_returns` is larger than 0. This will be our target variable.

In [6]:
# Load the dataset returns.csv and set the index to level_0 and time
# YOUR CODE HERE

returns_path = Path("returns.csv")
returns_df = pd.read_csv(returns_path)
returns_df1 = returns_df.set_index('level_0')

In [7]:
returns_df1

,level_1,F_1_m_returns,1_m_returns,5_m_returns,10_m_returns
level_0,,,,,
FB,2021-01-05 09:40:00-05:00,0.000814,0.000074,0.001260,0.004758
FB,2021-01-05 09:41:00-05:00,0.000887,0.000814,0.001889,0.004941
FB,2021-01-05 09:42:00-05:00,0.000628,0.000887,0.001999,0.003782
FB,2021-01-05 09:43:00-05:00,0.000480,0.000628,0.003408,0.007850
FB,2021-01-05 09:44:00-05:00,-0.001291,0.000480,0.002886,0.005416
...,...,...,...,...,...
TSLA,2021-01-05 15:55:00-05:00,-0.000259,0.000437,0.001140,0.000164
TSLA,2021-01-05 15:56:00-05:00,0.002033,-0.000259,0.000382,0.000806
TSLA,2021-01-05 15:57:00-05:00,0.000463,0.002033,0.003463,0.003868


In [8]:
# Create a separate dataframe for features and define the target variable as a binary target
# YOUR CODE HERE
X = returns_df1[['F_1_m_returns','1_m_returns','5_m_returns','10_m_returns']]
X.head()

,F_1_m_returns,1_m_returns,5_m_returns,10_m_returns
level_0,,,,
FB,0.000814,0.000074,0.001260,0.004758
FB,0.000887,0.000814,0.001889,0.004941
FB,0.000628,0.000887,0.001999,0.003782
FB,0.000480,0.000628,0.003408,0.007850
FB,-0.001291,0.000480,0.002886,0.005416


In [9]:
# Create the target variable
# YOUR CODE HERE

returns_df1["signal"] = 0.0
returns_df1.loc[(returns_df1["F_1_m_returns"] >= 0), "signal"] = 1
returns_df1.loc[(returns_df1["F_1_m_returns"] < 0), "signal"] = 0
returns_df1

,level_1,F_1_m_returns,1_m_returns,5_m_returns,10_m_returns,signal
level_0,,,,,,
FB,2021-01-05 09:40:00-05:00,0.000814,0.000074,0.001260,0.004758,1.0
FB,2021-01-05 09:41:00-05:00,0.000887,0.000814,0.001889,0.004941,1.0
FB,2021-01-05 09:42:00-05:00,0.000628,0.000887,0.001999,0.003782,1.0
FB,2021-01-05 09:43:00-05:00,0.000480,0.000628,0.003408,0.007850,1.0
FB,2021-01-05 09:44:00-05:00,-0.001291,0.000480,0.002886,0.005416,0.0
...,...,...,...,...,...,...
TSLA,2021-01-05 15:55:00-05:00,-0.000259,0.000437,0.001140,0.000164,0.0
TSLA,2021-01-05 15:56:00-05:00,0.002033,-0.000259,0.000382,0.000806,1.0
TSLA,2021-01-05 15:57:00-05:00,0.000463,0.002033,0.003463,0.003868,1.0


In [10]:
y = returns_df1[['signal']]
y.head(10)

,signal
level_0,
FB,1.0
FB,1.0
FB,1.0
FB,1.0
FB,0.0
FB,0.0
FB,0.0
FB,1.0
FB,1.0


##### Note:
> Notice that we don't use shuffle when splitting the dataset into a training and testing dataset. 

> We want to keep the original ordering of the data, so we don't end up using observations in the future to predict past observations,

> This is a critical mistake known as look ahead bias.

#### 2. Use the train_test_split library to split the dataset into a training and testing dataset, with 70% used for testing
* Set the shuffle parameter to False, so that you use the first 70% for training to prvent look ahead bias.
* Make sure you have these 4 variables: `X_train`, `X_test`, `y_train`, `y_test`. 

In [11]:
# Import train_test_split 
from sklearn.model_selection import train_test_split

# Split the dataset without shuffling
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=False)

#### 3. Use the `Counter` function to test the distribution of the data. 
* The result of `Counter({1: 668, 0: 1194})` reveals the data is indeed unbalanced.

In [12]:
# Import the Counter function from the collections library
from collections import Counter

# Use Counter to count the number 1s and 0 in y_train
# YOUR CODE HERE
y_train.value_counts()

signal
1.0       1251
0.0        744
dtype: int64

#### 4. Balance the dataset with the Oversampler libary, setting `random state= 1`.

In [13]:
# Import RandomOverSampler from the imblearn library
from imblearn.over_sampling import RandomOverSampler

# Use RandomOverSampler to resample the datase using random_state=1
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

#### 5. Test the distribution once again with `Counter`. The new result of `Counter({1: 1194, 0: 1194})` shows the data is now balanced.

In [14]:
# Use Counter again to verify imbalance removed
# YOUR CODE HERE
y_resampled.value_counts()

signal
0.0       1251
1.0       1251
dtype: int64

# Machine Learning

#### 1. The first cells in this section provide an example of how to fit and train your model using the `LogisticRegression` model from sklearn:
* Import select model.
* Instantiate model object.
* Fit the model to the resampled data - `X_resampled` and `y_resampled`.
* Predict the model using `X_test`.
* Print the classification report.

In [15]:
# Import classification_report from sklearn
from sklearn.metrics import classification_report

In [16]:
# Import LogisticRegression from sklearn
from sklearn.linear_model import LogisticRegression

# Create a LogisticRegression model and train it on the X_resampled data we created before
log_model = LogisticRegression()
log_model.fit(X_resampled, y_resampled)  

# Use the model you trained to predict using X_test
y_pred = log_model.predict(X_test)   

# Print out a classification report toevaluate performance
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

         0.0     0.7782    0.7427    0.7600       307
         1.0     0.7876    0.8184    0.8027       358

    accuracy                         0.7835       665
   macro avg     0.7829    0.7806    0.7814       665
weighted avg     0.7833    0.7835    0.7830       665



C:\Users\antho\anaconda3\envs\algotrading\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### 2. Use the same approach as above to train and test the following ML Algorithms:
* [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
* [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)
* [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)
* [XGBClassifier](https://xgboost.readthedocs.io/en/latest/python/python_api.html)

#### RandomForestClassifier

In [17]:
# Import RandomForestClassifier from sklearn
from sklearn.ensemble import RandomForestClassifier

# Create a RandomForestClassifier model and train it on the X_resampled data we created before
# YOUR CODE HERE
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
rf_model = rf_model.fit(X_resampled, y_resampled)

# Use the model you trained to predict using X_test
# YOUR CODE HERE  
predictions = rf_model.predict(X_test)

# Print out a classification report to evaluate performance
# YOUR CODE HERE
print("Classification Report")
print(classification_report(y_test, predictions))

C:\Users\antho\anaconda3\envs\algotrading\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


Classification Report
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       307
         1.0       1.00      1.00      1.00       358

    accuracy                           1.00       665
   macro avg       1.00      1.00      1.00       665
weighted avg       1.00      1.00      1.00       665



#### GradientBoostingClassifier

In [18]:
# Import RandomForestClassifier from sklearn
from sklearn.ensemble import GradientBoostingClassifier

# Create a GradientBoostingClassifier model and train it on the X_resampled data we created before
# YOUR CODE HERE

model_gbc = GradientBoostingClassifier(n_estimators=500, learning_rate=1.0, 
                                 max_depth=1, random_state=0).fit(X_resampled, y_resampled)

model_gbc.fit(X_resampled, y_resampled)

# Use the model you trained to predict using X_test
# YOUR CODE HERE     

predictions_gbc = model_gbc.predict(X_test)

# Print out a classification report to evaluate performance
# YOUR CODE HERE
print("Classification Report")
print(classification_report(y_test, predictions_gbc))

C:\Users\antho\anaconda3\envs\algotrading\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\antho\anaconda3\envs\algotrading\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Classification Report
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       307
         1.0       1.00      1.00      1.00       358

    accuracy                           1.00       665
   macro avg       1.00      1.00      1.00       665
weighted avg       1.00      1.00      1.00       665



#### AdaBoostClassifier

In [19]:
# Import RandomForestClassifier from sklearn
from sklearn.ensemble import AdaBoostClassifier

# Create a AdaBoostClassifier model and train it on the X_resampled data we created before
# YOUR CODE HERE

model_ada = AdaBoostClassifier(n_estimators=100, random_state=0)

model_ada.fit(X_resampled, y_resampled)

# Use the model you trained to predict using X_test
# YOUR CODE HERE

predictions_ada = model_ada.predict(X_test)

# Print out a classification report to evaluate performance
# YOUR CODE HERE
print("Classification Report")
print(classification_report(y_test, predictions_ada))

Classification Report
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       307
         1.0       1.00      1.00      1.00       358

    accuracy                           1.00       665
   macro avg       1.00      1.00      1.00       665
weighted avg       1.00      1.00      1.00       665



C:\Users\antho\anaconda3\envs\algotrading\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### XGBClassifier

In [20]:
# Import RandomForestClassifier from sklearn
from xgboost import XGBClassifier

# Create a XGBClassifier model and train it on the X_resampled data we created before
# YOUR CODE HERE

model_xgb =  XGBClassifier(n_estimators=100, random_state=0)
model_xgb.fit(X_resampled, y_resampled)

# Use the model you trained to predict using X_test
# YOUR CODE HERE
predictions_xgb = model_xgb.predict(X_test)

# Print out a classification report to evaluate performance
# YOUR CODE HERE

print("Classification Report")
print(classification_report(y_test, predictions_xgb))

[19:49:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Classification Report
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       307
         1.0       1.00      1.00      1.00       358

    accuracy                           1.00       665
   macro avg       1.00      1.00      1.00       665
weighted avg       1.00      1.00      1.00       665



C:\Users\antho\anaconda3\envs\algotrading\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\antho\anaconda3\envs\algotrading\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\antho\anaconda3\envs\algotrading\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using 

In [21]:
# Import RandomForestClassifier from sklearn
from sklearn.ensemble import AdaBoostClassifier

# Create a AdaBoostClassifier model and train it on the X_resampled data we created before
# YOUR CODE HERE

model_ada = AdaBoostClassifier(n_estimators=100, random_state=0)

model_ada.fit(X_resampled, y_resampled)

# Use the model you trained to predict using X_test
# YOUR CODE HERE

predictions_ada = model_ada.predict(X_test)

# Print out a classification report to evaluate performance
# YOUR CODE HERE
print("Classification Report")
print(classification_report(y_test, predictions_ada))

Classification Report
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       307
         1.0       1.00      1.00      1.00       358

    accuracy                           1.00       665
   macro avg       1.00      1.00      1.00       665
weighted avg       1.00      1.00      1.00       665



C:\Users\antho\anaconda3\envs\algotrading\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Evaluate the performance of each model


#### 1. Using the classification report for each model, choose the model with the highest precision for use in your algo-trading program.
#### 2. Save the selected model with the `joblib` libary to avoid retraining every time you wish to use it.

In [22]:
# Import the joblib library 
import joblib

# Use the library to save the model that you want to use for trading
joblib.dump(log_model, 'log_model.pkl')

['log_model.pkl']

## Part 3: Implement the strongest model using Apaca API

### Develop the Algorithm


#### 1. Use the provided code to ping the Alpaca API and create the DataFrame needed to feed data into the model.
   * This code will also store the correct feature data in `X` for later use.

In [29]:
# Create the list of tickers

ticker_list = ['FB','AMZN','AAPL','NFLX', 'GOOGL', 'MSFT', 'TSLA']
# Define Dates

beg_date = '2021-01-06'
end_date = '2021-01-06'

# Convert the date in a format the Alpaca API reqires
start =  pd.Timestamp(f'{beg_date} 09:30:00-0400', tz='America/New_York').replace(hour=9, minute=30, second=0).astimezone('GMT').isoformat()[:-6]+'Z'
end   =  pd.Timestamp(f'{end_date} 16:00:00-0400', tz='America/New_York').replace(hour=15, minute=0, second=0).astimezone('GMT').isoformat()[:-6]+'Z'
timeframe='1Min'

# Use iloc to get the last 10 mins every time we pull new data
prices = api.get_bars(ticker_list, timeframe, start=start, end=end).df.iloc[-11:]
prices.ffill(inplace=True)   

# Create an empty DataFrame for closing prices
df_closing_prices = pd.DataFrame()

# Fetch the closing prices of our tickers
df_closing_prices["FB"] = prices["FB"]["close"]
df_closing_prices["AMZN"] = prices["AMZN"]["close"]
df_closing_prices["AAPL"] = prices["AAPL"]["close"]
df_closing_prices["NFLX"] = prices["NFLX"]["close"]
df_closing_prices["GOOGL"] = prices["GOOGL"]["close"]
df_closing_prices['MSFT'] = prices['MSFT']["close"]
df_closing_prices['TSLA'] = prices['TSLA']["close"]

print(df_closing_prices.head(20))

KeyError: 'FB'

In [26]:
# Fetch the closing prices of our tickers
df_closing_prices["FB"] = prices["FB"]["close"]
df_closing_prices["AMZN"] = prices["AMZN"]["close"]
df_closing_prices["AAPL"] = prices["AAPL"]["close"]
df_closing_prices["NFLX"] = prices["NFLX"]["close"]
df_closing_prices["GOOGL"] = prices["GOOGL"]["close"]
df_closing_prices['MSFT'] = prices['MSFT']["close"]
df_closing_prices['TSLA'] = prices['TSLA']["close"]

print(df_closing_prices.head(20))

KeyError: 'FB'

In [ ]:
# Create list of momentums
list_of_momentums = [1,5,10]

for i in list_of_momentums:  
    # Compute percentage change for each one of the momentums in the momentum list
    returns_temp = df_closing_prices.pct_change(i)
    # Unstack the returns 
    returns_temp = pd.DataFrame(returns_temp.unstack())
    name = f'{i}_m_returns'
    returns_temp.rename(columns={0: name}, inplace = True)
    # Reset the index so we can merge based on index
    returns_temp.reset_index(inplace = True)
    # Merge newly computed returns with previously created returns
    if i ==1:
        returns = returns_temp
    else:
        returns = pd.merge(returns,returns_temp,left_on=['level_0', 'time'],right_on=['level_0', 'time'], how='left', suffixes=('_original', 'right'))

# Drop nulls and set index
returns.dropna(axis=0, how='any', inplace=True)
returns.set_index(['level_0', 'time'], inplace=True)

# Generate feature data and preview first 10 rows.
X = returns
X.head(10)

#### 2. Using `joblib`, load the chosen model.

In [ ]:
# Load the previously trained and saved model using joblib
# YOUR CODE HERE

#### 3. Use the model file to make predicttions:
* Use `predict` on `X` and save this as `y_pred`.
* Convert `y_pred` to a DataFrame, setting the index to the index of `X`.
* Rename the column 0 to 'buy', be sure to set `inplace =True`.

In [ ]:
# Use the model file to predict on X
# YOUR CODE HERE

# Convert y_pred to a dataframe, set the index to the index of X
# YOUR CODE HERE

# Rename the column 0 to 'buy', be sure to set inplace =True
# YOUR CODE HERE

#### 4. Filter the stocks where 'buy' is equal to 1, saving the filter as `y_pred`.

In [ ]:
# Filter the stocks where 'buy' is equal to 1
# YOUR CODE HERE

#### 5. Using the `y_pred` filter, create a dictionary called `buy_dict` and assign 'n' to each Ticker (key value) as a placeholder.

In [ ]:
# Create dictionary from y_pred and assign a 'n' to each of them for now as a placeholder.
buy_dict = dict.fromkeys(y_pred.index.get_level_values(0), 'n')
buy_dict

#### 6. Obtain the total available equity in your account from the Alpaca API and store in a variable called `total_capital`. You will split the capital equally between all selected stocks per the CIO's request.

In [ ]:
# Pull the total available equity in our account from the  Alpaca API
# YOUR CODE HERE

In [ ]:
# Compute capital per stock, divide equity in account by number of stocks
# Use Alpaca API to pull the equity in the account
if len(buy_dict) > 0:
    capital_per_stock = float(total_capital)/ len(buy_dict)
else:
    capital_per_stock = 0
print(f'Capital per stock: {capital_per_stock}')

#### 7. Use a for-loop to iterate through `buy_dict` to determine the number stocks you need to buy for each ticker.

In [ ]:
# Use for loop to iterate through dictionary of buys 
# Determine the number stocks we need to buy for each ticker
for ticker in buy_dict:
    try:
        buy_dict[ticker] = int(capital_per_stock /int(prices[ticker].iloc[-1]['close']))
    except:
        pass

print(buy_dict)

#### 8. Cancel all previous orders in the Alpaca API (so you don't buy more than intended) and sell all currently held stocks to close all positions.

In [ ]:
# Cancel all previous orders in the Alpaca API
# YOUR CODE HERE

# Sell all currently held stocks to close all positions
# YOUR CODE HERE

#### 9. Iterate through `buy_dict` and send a buy order for each ticker with their corresponding number of shares.

In [ ]:
# Iterate through the longlist object and send a buy order for each ticker with a corresponding number of shares:
# YOUR CODE HERE

### Automate the algorithm

#### 1. Make a function called `trade()` that incorporates all of the steps above.

In [ ]:
# Add all of the steps conducted above into the function trade
def trade():

    ticker_list = ['FB','AMZN','AAPL','NFLX', 'GOOGL', 'MSFT', 'TSLA']
    # Notice that we remove the start and end variables since we want the latest prices.
    timeframe='1Min'
    # Use iloc to get the last 10 mins every time we pull new data
    prices = api.get_barset(ticker_list, "minute").df.iloc[-11:]
    prices.ffill(inplace=True)   

    # Create and empty DataFrame for closing prices
    df_closing_prices = pd.DataFrame()

    # Fetch the closing prices of our tickers
    df_closing_prices["FB"] = prices["FB"]["close"]
    df_closing_prices["AMZN"] = prices["AMZN"]["close"]
    df_closing_prices["AAPL"] = prices["AAPL"]["close"]
    df_closing_prices["NFLX"] = prices["NFLX"]["close"]
    df_closing_prices["GOOGL"] = prices["GOOGL"]["close"]
    df_closing_prices['MSFT'] = prices['MSFT']["close"]
    df_closing_prices['TSLA'] = prices['TSLA']["close"]
    print(df_closing_prices.head())
    
    # Loop through momentums to build new DataFrame
    list_of_momentums = [1,5,10]
    for i in list_of_momentums:   
        returns_temp = df_closing_prices.pct_change(i)
        returns_temp = pd.DataFrame(returns_temp.unstack())
        name = f'{i}_m_returns'
        returns_temp.rename(columns={0: name}, inplace = True)
        returns_temp.reset_index(inplace = True)
        if i ==1:
            returns = returns_temp
        else:
            returns = pd.merge(returns,returns_temp,left_on=['level_0', 'time'],right_on=['level_0', 'time'], how='left', suffixes=('_original', 'right'))

    # Drop nulls and set index            
    returns.dropna(axis=0, how='any', inplace=True)
    returns.set_index(['level_0', 'time'], inplace=True)

    # Preprocess data for model
    # YOUR CODE HERE

    # Create the `buy_dict` object
    # YOUR CODE HERE
    
    # Split capital between stocks and determine buy or sell
    # YOUR CODE HERE

    
    # Cancel pending orders and close positions
    # YOUR CODE HERE
   
    
    # Submit orders
    # YOUR CODE HERE


#### 2. Import Python's schedule module.

In [ ]:
# Import Python's schedule module 
# YOUR CODE HERE

#### 3. Use the "schedule" module to automate the algorithm:
* Clear the schedule with `.clear()`.
* Define a schedule to run the trade function every minute at 5 seconds past the minute mark (e.g. `10:31:05`).
* Use the Alpaca API to check whether the market is open.
* Use run_pending() function inside schedule to execute the schedule you defined while the market is open

In [ ]:
# Clear the schedule
# YOUR CODE HERE

# Define a schedule to run the trade function every minute at 5 seconds past the minute mark (e.g. 10:31:05)
# YOUR CODE HERE

# Use the Alpaca API to check whether the market is open
# YOUR CODE HERE

# Use run_pending() function inside schedule to execute the schedule you defined as long as the market is open
# YOUR CODE HERE